In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

######  1. Write a python program which searches all the product under a particular product from www.amazon.in. The
######   product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
######   guitars.

In [98]:
import requests
from bs4 import BeautifulSoup

def search_amazon_product(product_name):
    base_url = "https://www.amazon.in/s?k=guitar"

    response = requests.get(base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        product_list = soup.find_all('div', class_='s-result-item')

        if not product_list:
            print("No products found.")
            return

        for product in product_list:
            product_title = product.find('span', class_='a-text-normal')
            product_price = product.find('span', class_='a-offscreen')
            
            if product_title and product_price:
                print(f"Product: {product_title.text.strip()}")
                print(f"Price: {product_price.text.strip()}")
                print("-" * 30)
    else:
        print(f"Error: {response.status_code}")

if __name__ == "__main__":
    user_input = input("Enter the product you want to search on Amazon: ")
    search_amazon_product(user_input)


Enter the product you want to search on Amazon: guitar
Product: Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black
Price: ₹1,999
------------------------------
Product: Tanglewood Crossroads TWCR DCE Semi Acoustic Guitar, 6 Strings, Dreadnought Cutaway, Whiskey Barrel Burst Satin Finish, With Padded Guitar Bag
Price: ₹14,499
------------------------------
Product: Intern INT-38C Acoustic Guitar Kit (Blue)
Price: ₹2,129
------------------------------
Product: Juarez Lindenwood Acoustic Guitar Kit, 38 Inches Cutaway, 38C With Bag, Strings, Pick And Strap, 3TS Sunburst
Price: ₹1,999
------------------------------
Product: Intern INT-38C-NT-G 38 Incheses lindenwood Right HAnded Acoustic Guitar Kit With Bag, Strings, Pick And Strap (Natural, 6-Strings)
Price: ₹2,399
------------------------------
Product: Yamaha F280 Acoustic Rosewood Guitar (Natural, Beige)
Price: ₹7,499
------------------------------
Product: Fender Squier 39 Inch 6 String Cutaway Acous

###### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search
###### results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then
###### scrape all the products available under that product name. Details to be scraped are: "Brand
###### Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
###### “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_product_details(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(product_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        brand_name = soup.find('a', {'id': 'bylineInfo'})
        brand_name = brand_name.text.strip() if brand_name else "Not available"

        product_title = soup.find('span', {'id': 'productTitle'})
        product_title = product_title.text.strip() if product_title else "Not available"

        product_price = soup.find('span', {'id': 'priceblock_ourprice'})
        product_price = product_price.text.strip() if product_price else "Not available"

        return_exchange = soup.find('div', {'data-asin': product_url.split("/")[-1]})
        return_exchange = return_exchange.text.strip() if return_exchange else "Not available"

        expected_delivery = soup.find('div', {'id': 'ddmDeliveryMessage'})
        expected_delivery = expected_delivery.text.strip() if expected_delivery else "Not available"

        availability = soup.find('div', {'id': 'availability'})
        availability = availability.text.strip() if availability else "Not available"

        return {
            "Brand Name": brand_name,
            "Name of the Product": product_title,
            "Price": product_price,
            "Return/Exchange": return_exchange,
            "Expected Delivery": expected_delivery,
            "Availability": availability,
            "Product URL": product_url
        }
    else:
        print(f"Error fetching details for {product_url}: {response.status_code}")
        return None

def search_amazon_product(product_name, max_pages=3):
    base_url = "https://www.amazon.in/s"
    params = {
        "k": product_name
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    all_products_data = []

    for page in range(1, max_pages + 1):
        params["page"] = page

        response = requests.get(base_url, params=params, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            product_list = soup.find_all('div', class_='s-result-item')

            if not product_list:
                print("No products found.")
                break

            for product in product_list:
                product_title = product.find('span', class_='a-text-normal')
                product_url = product.find('a', {'class': 'a-link-normal'}, href=True)

                if product_title and product_url:
                    product_url = "https://www.amazon.in" + product_url['href']
                    product_details = get_product_details(product_url)

                    if product_details:
                        all_products_data.append(product_details)

    if all_products_data:
        df = pd.DataFrame(all_products_data)
        df.to_csv(f"{product_name}_products_data.csv", index=False)
        print("Data saved successfully.")
    else:
        print("No data to save.")

if __name__ == "__main__":
    user_input = input("Enter the product you want to search on Amazon: ")
    search_amazon_product(user_input)


###### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [ ]:
import selenium

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

def scrape_images(query, num_images=10):
    driver = webdriver.Chrome()  # You need to have the ChromeDriver executable in your PATH or provide the path.
    driver.get("https://images.google.com")

    search_box = driver.find_element("name", "q")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)

    time.sleep(2)  # Allow time for the results to load

    # Scroll to load more images (adjust as needed)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Get image URLs
    image_elements = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
    image_urls = [element.get_attribute("src") for element in image_elements[:num_images]]

    driver.quit()

    return image_urls

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

    for keyword in keywords:
        print(f"Scraping images for '{keyword}'")
        images = scrape_images(keyword, num_images=10)
        for i, image_url in enumerate(images, start=1):
            print(f"{i}. {image_url}")

Scraping images for 'fruits'


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=118.0.5993.118)
Stacktrace:
	GetHandleVerifier [0x00007FF6077A8EF2+54786]
	(No symbol) [0x00007FF607715612]
	(No symbol) [0x00007FF6075CA4FD]
	(No symbol) [0x00007FF607606C17]
	(No symbol) [0x00007FF607605F84]
	(No symbol) [0x00007FF60762BE6A]
	(No symbol) [0x00007FF6076022E6]
	(No symbol) [0x00007FF60762C080]
	(No symbol) [0x00007FF607644D02]
	(No symbol) [0x00007FF60762BC43]
	(No symbol) [0x00007FF607600941]
	(No symbol) [0x00007FF607601B84]
	GetHandleVerifier [0x00007FF607AF7F52+3524194]
	GetHandleVerifier [0x00007FF607B4D800+3874576]
	GetHandleVerifier [0x00007FF607B45D7F+3843215]
	GetHandleVerifier [0x00007FF607845086+694166]
	(No symbol) [0x00007FF607720A88]
	(No symbol) [0x00007FF60771CA94]
	(No symbol) [0x00007FF60771CBC2]
	(No symbol) [0x00007FF60770CC83]
	BaseThreadInitThunk [0x00007FF932537344+20]
	RtlUserThreadStart [0x00007FF932EC26B1+33]


In [17]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_smartphone_details(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(product_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        brand_name = soup.find('span', {'class': '_1iWCFp'})
        brand_name = brand_name.text.strip() if brand_name else "-"

        smartphone_name = soup.find('span', {'class': 'B_NuCI'})
        smartphone_name = smartphone_name.text.strip() if smartphone_name else "-"

        color = soup.find('div', {'class': '_1AtVbE'})
        color = color.text.strip() if color else "-"

        ram = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'RAM' in x)
        ram = ram.text.strip() if ram else "-"

        storage = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'Internal Storage' in x)
        storage = storage.text.strip() if storage else "-"

        primary_camera = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'Primary Camera' in x)
        primary_camera = primary_camera.text.strip() if primary_camera else "-"

        secondary_camera = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'Secondary Camera' in x)
        secondary_camera = secondary_camera.text.strip() if secondary_camera else "-"

        display_size = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'Display Size' in x)
        display_size = display_size.text.strip() if display_size else "-"

        battery_capacity = soup.find('div', {'class': '_1AtVbE'}, string=lambda x: 'Battery Capacity' in x)
        battery_capacity = battery_capacity.text.strip() if battery_capacity else "-"

        price = soup.find('div', {'class': '_30jeq3 _16Jk6d'})
        price = price.text.strip() if price else "-"

        return {
            "Brand Name": brand_name,
            "Smartphone Name": smartphone_name,
            "Colour": color,
            "RAM": ram,
            "Storage(ROM)": storage,
            "Primary Camera": primary_camera,
            "Secondary Camera": secondary_camera,
            "Display Size": display_size,
            "Battery Capacity": battery_capacity,
            "Price": price,
            "Product URL": product_url
        }
    else:
        print(f"Error fetching details for {product_url}: {response.status_code}")
        return None

def search_flipkart_smartphones(query):
    base_url = "https://www.flipkart.com/search"
    params = {
        "q": query
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        product_list = soup.find_all('a', {'class': '_1fQZEK'})

        if not product_list:
            print("No products found.")
            return

        all_smartphones_data = []

        for product in product_list:
            product_url = "https://www.flipkart.com" + product['href']
            product_details = get_smartphone_details(product_url)

            if product_details:
                all_smartphones_data.append(product_details)

        if all_smartphones_data:
            df = pd.DataFrame(all_smartphones_data)
            df.to_csv(f"{query}_smartphones_data.csv", index=False)
            print("Data saved successfully.")
        else:
            print("No data to save.")
    else:
        print(f"Error searching for {query}: {response.status_code}")

if __name__ == "__main__":
    user_input = input("Enter the smartphone you want to search on Flipkart: ")
    search_flipkart_smartphones(user_input)


Enter the smartphone you want to search on Flipkart: samsung s22 ultra
Error searching for samsung s22 ultra: 500


#### 4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [31]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
import time
from selenium.webdriver.common.by import By

In [32]:
driver = webdriver.Chrome()
url4="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url4)

In [33]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [34]:
#scraping the Brand_Name 
BName=driver.find_elements(By.XPATH,"//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['SAMSUNG Galaxy F14 5G (GOAT Green, 128 GB)', 'vivo T2x 5G (Aurora Gold, 128 GB)', 'vivo T2x 5G (Glimmer Black, 128 GB)', 'vivo T2x 5G (Aurora Gold, 128 GB)', 'vivo T2x 5G (Glimmer Black, 128 GB)', 'vivo T2x 5G (Aurora Gold, 128 GB)', 'vivo T2x 5G (Marine Blue, 128 GB)', 'OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)', 'vivo T2x 5G (Marine Blue, 128 GB)', 'REDMI 12C (Matte Black, 64 GB)', 'REDMI 12C (Lavender Purple, 64 GB)', 'APPLE iPhone 14 (Blue, 128 GB)', 'REDMI 12C (Royal Blue, 64 GB)', 'APPLE iPhone 14 (Starlight, 128 GB)', 'SAMSUNG Galaxy F14 5G (OMG Black, 128 GB)', 'vivo T2x 5G (Glimmer Black, 128 GB)', 'REDMI 12 (Moonstone Silver, 128 GB)', 'SAMSUNG Galaxy F14 5G (OMG Black, 128 GB)', 'REDMI 12 (Pastel Blue, 128 GB)', 'REDMI 12 (Jade Black, 128 GB)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256 GB)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 128 GB)', 'POCO C51 (Royal Blue, 64 GB)', 'SAMSUNG Galaxy F14 5G (B.A.E. Purple, 128 GB)']


In [35]:
ram=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '4 GB RAM | 128 GB ROM', '4 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM', '4 GB RAM | 128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB']


In [36]:
#scraping the P_F_Camera 
PC=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

24 ['50MP + 2MP | 13MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '64MP Rear Camera | 16MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP Rear Camera | 5MP Front Camera', '50MP Rear Camera | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP Rear Camera | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 2MP | 13MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 8MP + 2MP | 8MP Front Camera', '50MP + 2MP | 13MP Front Camera', '50MP + 8MP + 2MP | 8MP Front Camera', '50MP + 8MP + 2MP | 8MP Front Camera', '108MP Rear Camera', '108MP Rear Camera', '8MP Dual Rear Camera | 5MP Front Camera', '50MP + 2MP | 13MP Front Camera']


In [37]:
#scraping the Display_size_Resolution 
DS=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

24 ['16.76 cm (6.6 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.74 cm (6.59 inch) Display', '16.71 cm (6.58 inch) Full HD+ Display', '17.04 cm (6.71 inch) HD+ Display', '17.04 cm (6.71 inch) HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '17.04 cm (6.71 inch) HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '17.07 cm (6.72 inch) Display', '17.07 cm (6.72 inch) Display', '16.56 cm (6.52 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display']


In [38]:
#scraping the Battery 
B=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

24 ['6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', 'A15 Bionic Chip, 6 Core Processor Processor', '5000 mAh Battery', 'A15 Bionic Chip, 6 Core Processor Processor', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery']


In [39]:
#scraping the ProcessorAndCores 
P=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

24 ['Exynos 1330, Octa Core Processor', 'Dimensity 6020 Processor', 'Dimensity 6020 Processor', 'Dimensity 6020 Processor', 'Dimensity 6020 Processor', 'Dimensity 6020 Processor', 'Dimensity 6020 Processor', '12 Months', 'Dimensity 6020 Processor', 'Helio G85 Processor', 'Helio G85 Processor', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', 'Helio G85 Processor', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', 'Exynos 1330, Octa Core Processor', 'Dimensity 6020 Processor', 'Helio G88 Processor', 'Exynos 1330, Octa Core Processor', 'Helio G88 Processor', 'Helio G88 Processor', 'Domestic Warranty of 12 months on phone & 6 months on accessories', 'Domestic Warranty of 12 months on phone & 6 months on accessories', 'Helio G36 Processor', 'Exynos 1330, Octa Core Processor']


In [40]:
#scraping the Price 
price=driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

24 ['₹14,990', '₹12,999', '₹14,999', '₹14,999', '₹11,999', '₹11,999', '₹14,999', '₹16,910', '₹11,999', '₹6,799', '₹6,799', '₹60,999', '₹6,799', '₹61,999', '₹13,990', '₹12,999', '₹10,499', '₹14,990', '₹9,299', '₹10,499', '₹21,699', '₹19,798', '₹6,249', '₹14,990']


In [41]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Amount P_F_Camera']=P_F_Camera
FlipKart['Display_size_Resolution']=Display_size_Resolution
FlipKart['ProcessorAndCores']=ProcessorAndCores
FlipKart['Battery']=Battery
FlipKart['Price']=Price

In [42]:
FlipKart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"SAMSUNG Galaxy F14 5G (GOAT Green, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 13MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,"Exynos 1330, Octa Core Processor",6000 mAh Battery,"₹14,990"
1,"vivo T2x 5G (Aurora Gold, 128 GB)",6 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹12,999"
2,"vivo T2x 5G (Glimmer Black, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹14,999"
3,"vivo T2x 5G (Aurora Gold, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹14,999"
4,"vivo T2x 5G (Glimmer Black, 128 GB)",4 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹11,999"
5,"vivo T2x 5G (Aurora Gold, 128 GB)",4 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹11,999"
6,"vivo T2x 5G (Marine Blue, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹14,999"
7,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",6 GB RAM | 128 GB ROM,64MP Rear Camera | 16MP Front Camera,16.74 cm (6.59 inch) Display,12 Months,5000 mAh Battery,"₹16,910"
8,"vivo T2x 5G (Marine Blue, 128 GB)",4 GB RAM | 128 GB ROM,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Dimensity 6020 Processor,5000 mAh Battery,"₹11,999"
9,"REDMI 12C (Matte Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP Rear Camera | 5MP Front Camera,17.04 cm (6.71 inch) HD+ Display,Helio G85 Processor,5000 mAh Battery,"₹6,799"


###  5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [22]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
import time
import re

In [23]:
driver = webdriver.Chrome()
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search=driver.find_element("id", "searchboxinput")                                             # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                    # entering values in search bar
# button = driver.find_element(By.XPATH,"searchbox-searchbutton")               # locating search button
button = driver.find_element("id","searchbox-searchbutton")
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Nagpur
URL Extracted:  https://www.google.co.in/maps/place/Nagpur,+Maharashtra,+India/@21.161006,78.9076921,11z/data=!3m1!4b1!4m6!3m5!1s0x3bd4c0a5a31faf13:0x19b37d06d0bb3e2b!8m2!3d21.1458004!4d79.0881546!16zL20vMDJjOTht?entry=ttu
Latitude = 21.161006, Longitude = 78.9076921


### 6.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [24]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
import time
import re

In [40]:
driver = webdriver.Chrome()
# opening google maps
driver.get("https://www.digit.in/top-products/top-10-laptops-5.html")                                          

In [41]:
B_Name=[]
Osystem =[]
Resolution=[]
Processor = []

In [42]:
#scraping the Brand_Name 
BName=driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]/a') 
for i in BName:
    if i.text is None :
        B_Name.append("--") 
    else:
        B_Name.append(i.text)
print(len(B_Name),B_Name)

6 ['Apple MacBook Air 2022 M2 MLY03HN/A', 'Samsung Galaxy Book3 Pro 360', 'Lenovo Yoga 9i13th Gen Core i7-1360P', 'Apple MacBook Pro M2 (2023-MPHK3HN/A)', 'Dell XPS 13 Plus D560075WIN9S 12th Gen Core i7-1260P', 'Acer Swift X SFX16-51G 11th Gen Core i7-11390H']


In [43]:
Digit=pd.DataFrame([])
Digit['Brand_Name']=B_Name

In [44]:
B_Name

['Apple MacBook Air 2022 M2 MLY03HN/A',
 'Samsung Galaxy Book3 Pro 360',
 'Lenovo Yoga 9i13th Gen Core i7-1360P',
 'Apple MacBook Pro M2 (2023-MPHK3HN/A)',
 'Dell XPS 13 Plus D560075WIN9S 12th Gen Core i7-1260P',
 'Acer Swift X SFX16-51G 11th Gen Core i7-11390H']

### 7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [58]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
import time
import re

In [59]:
driver = webdriver.Chrome()
# opening google maps
driver.get("https://www.forbes.com/billionaires/")                                          

In [60]:
B_Rank=[]
B_Name =[]
B_Net_worth=[]
B_age = []
B_citizen=[]
B_source=[]
B_industry=[]

In [61]:
#scraping the Brand_Name 
Rank=driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]/a') 
for i in Rank:
    if i.text is None :
        B_Rank.append("--") 
    else:
        B_Rank.append(i.text)
print(len(B_Rank),B_Rank)

0 []


### 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted  from any YouTube Video.

In [57]:
driver = webdriver.Chrome( )  # Replace with the path to your WebDriver executable

# Open the YouTube video
video_url = 'https://www.youtube.com/watch?v=your_video_id'  # Replace with the URL of the YouTube video
driver.get(video_url)

# Scroll to load comments
scroll_pause_time = 2  # Adjust the pause time as needed
scrolls = 10  # Adjust the number of scrolls as needed

for _ in range(scrolls):
  driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
  time.sleep(scroll_pause_time)

# Extract comments, upvotes, and time
comments = driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]')
upvotes = driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
times = driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')

# Store the extracted data
extracted_data = []
for comment, upvote, time in zip(comments, upvotes, times):
  extracted_data.append({
  'comment': comment.text,
  'upvote': upvote.text,
  'time': time.text
  })

# Close the WebDriver
driver.quit()

# Print the extracted data
for data in extracted_data:
  print(data)

### 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [65]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the website
url = "https://www.hostelworld.com/hostels/London"
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find all the hostel listings on the page
hostel_listings = soup.find_all("div", class_="fabresult")

# Iterate over each hostel listing and extract the required information
for hostel in hostel_listings:
  # Extract hostel name
  name = hostel.find("h2", class_="title").text.strip()

  # Extract distance from city centre
  distance = hostel.find("span", class_="distance").text.strip()

  # Extract ratings
  ratings = hostel.find("div", class_="score orange").text.strip()

  # Extract total reviews
  total_reviews = hostel.find("div", class_="reviews").text.strip()

  # Extract overall reviews
  overall_reviews = hostel.find("div", class_="rating").text.strip()

  # Extract privates from price
  privates_price = hostel.find("div", class_="price").text.strip()

  # Extract dorms from price
  dorms_price = hostel.find("div", class_="price dorms").text.strip()

  # Extract facilities
  facilities = [facility.text.strip() for facility in hostel.find_all("span", class_="facilities")]

  # Extract property description
  description = hostel.find("div", class_="desc").text.strip()

  # Print the extracted information
  print("Name:", name)
  print("Distance from city centre:", distance)
  print("Ratings:", ratings)
  print("Total reviews:", total_reviews)
  print("Overall reviews:", overall_reviews)
  print("Privates from price:", privates_price)
  print("Dorms from price:", dorms_price)
  print("Facilities:", facilities)
  print("Description:", description)
  print()